# This script processing images and training your own  face classifier.

In [1]:

import tensorflow as tf
import numpy as np
import cv2

import os
from os.path import join as pjoin
import sys
import copy
import detect_face
import nn4 as network
import matplotlib.pyplot as plt


import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics  
from sklearn.externals import joblib
from sklearn.multioutput import MultiOutputClassifier
%pylab inline



Populating the interactive namespace from numpy and matplotlib


/home/avirup171/anaconda3/envs/tfenv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/avirup171/anaconda3/envs/tfenv/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
#face detection parameters
minsize = 20 # minimum size of face
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
factor = 0.709 # scale factor

#facenet embedding parameters

model_dir='./20180408-102900'#"Directory containing the graph definition and checkpoint files.")
model_def= 'nn4'  # "Points to a module containing the definition of the inference graph.")
image_size=96 #"Image size (height, width) in pixels."
pool_type='MAX' #"The type of pooling to use for some of the inception layers {'MAX', 'L2'}.
use_lrn=False #"Enables Local Response Normalization after the first layers of the inception network."
seed=42,# "Random seed."
batch_size= None # "Number of images to process in a batch."



# Creating Networks and Loading Parameters

In [3]:
gpu_memory_fraction=1.0
print(tf.__version__)
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, './model_check_point/')
        
      

1.13.1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


# Face Embedding

In [4]:
tf.reset_default_graph()
tf.Graph().as_default()
sess = tf.Session()
images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, 
                                                       image_size, 
                                                       image_size, 3), name='input')

phase_train_placeholder = tf.placeholder(tf.bool, name='phase_train')



embeddings = network.inference(images_placeholder, 1.0, 
                               phase_train=phase_train_placeholder,weight_decay=0.0)



saver = tf.train.import_meta_graph('./20180408-102900/model-20180408-102900.meta')

model_checkpoint_path='./20180408-102900/model-20180408-102900.ckpt-90'

saver.restore(sess, model_checkpoint_path)
print('facenet embedding')

name =  incept3a
inputSize =  192
kernelSize = {3,5}
kernelStride = {1,1}
outputSize = {128,32}
reduceSize = {96,16,32,64}
pooling = {MAX, 3, 3, 1, 1}
outputSize =  256

name =  incept3b
inputSize =  256
kernelSize = {3,5}
kernelStride = {1,1}
outputSize = {128,64}
reduceSize = {96,32,64,64}
pooling = {MAX, 3, 3, 1, 1}
outputSize =  320

name =  incept3c
inputSize =  320
kernelSize = {3,5}
kernelStride = {2,2}
outputSize = {256,64}
reduceSize = {128,32,0,0}
pooling = {MAX, 3, 3, 2, 2}
outputSize =  640

name =  incept4a
inputSize =  640
kernelSize = {3,5}
kernelStride = {1,1}
outputSize = {192,64}
reduceSize = {96,32,128,256}
pooling = {MAX, 3, 3, 1, 1}
outputSize =  640

name =  incept4b
inputSize =  640
kernelSize = {3,5}
kernelStride = {1,1}
outputSize = {224,64}
reduceSize = {112,32,128,224}
pooling = {MAX, 3, 3, 1, 1}
outputSize =  640

name =  incept4c
inputSize =  640
kernelSize = {3,5}
kernelStride = {1,1}
outputSize = {256,64}
reduceSize = {128,32,128,192}
pooling = {MAX, 3, 3

# reading training images from train folder


In [5]:
###### train_dir containing one subdirectory per image class 

data_dir=r'/home/avirup171/Documents/Face-Recognition/train_dir'#your own train folder

In [6]:
def to_rgb(img):
    w, h = img.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, 0] = ret[:, :, 1] = ret[:, :, 2] = img
    return ret

def read_img(person_dir,f):
    img=cv2.imread(pjoin(person_dir, f))
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    if gray.ndim == 2:
        img = to_rgb(gray)
    return img

def load_data(data_dir):
    data = {}
    pics_ctr = 0
    for guy in os.listdir(data_dir):
        person_dir = pjoin(data_dir, guy)
        
        curr_pics = [read_img(person_dir, f) for f in os.listdir(person_dir)]
        
     
        
        data[guy] = curr_pics
        
    return data

In [7]:
data=load_data(data_dir)
keys=[]
for key in data.items():
    keys.append(key)
    
for j in range(len(keys)):
    
    print('foler:{},image numbers：{}' .format([i[0] for i in keys][j],[len(i[1]) for i in keys][j]))
    

foler:Bibhas,image numbers：3
foler:Payel,image numbers：15
foler:Mallika,image numbers：11
foler:Proteem,image numbers：14


In [8]:
train_x=[]
train_y=[]
for j in range(len(keys)):
    
    for x in [i[1] for i in keys][j]:
       
        bounding_boxes, _ = detect_face.detect_face(x, minsize, pnet, rnet, onet, threshold, factor)
        nrof_faces = bounding_boxes.shape[0]#number of faces
        



        for face_position in bounding_boxes:
            face_position=face_position.astype(int)
            #print(face_position[0:4])
            cv2.rectangle(x, (face_position[0], face_position[1]), (face_position[2], face_position[3]), (0, 255, 0), 2)
            crop=x[face_position[1]:face_position[3],
                 face_position[0]:face_position[2],]

            crop = cv2.resize(crop, (96, 96), interpolation=cv2.INTER_CUBIC )

            #print(crop.shape)

            crop_data=crop.reshape(-1,96,96,3)
            #print(crop_data.shape)
            

            #emb_data = sess.run(embeddings, 
                                #feed_dict={images_placeholder: np.array(crop_data), phase_train_placeholder: True})[0]


            train_x.append(np.array(crop_data))
            train_y.append(0)
    print(len(train_x))

print('Length of train_x：{}'.format(len(train_x)))

3
21
34
49
Length of train_x：49


In [9]:
#train/test split
train_x=np.array(train_x)
train_x=train_x.reshape(-1,512)
train_y=np.array(train_x)
print(train_x.shape)
print(train_y.shape)


X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=.2, random_state=1)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(2646, 512)
(2646, 512)
(2116, 512) (2116, 512) (530, 512) (530, 512)


In [ ]:
# KNN Classifier  
from sklearn.neighbors import KNeighborsClassifier  
def knn_classifier(train_x, train_y):  
    classifier = KNeighborsClassifier(n_neighbors=3) 
    model=MultiOutputClassifier(classifier,n_jobs=-1)
    model.fit(train_x, train_y)  
    return model  

classifiers = knn_classifier 

model = classifiers(X_train,y_train)  
predict = model.predict(X_test)  

accuracy = metrics.accuracy_score(y_test, predict)  
print ('accuracy: %.2f%%' % (100 * accuracy)  ) 
  
    
#save model
joblib.dump(model, './model_check_point/knn_classifier.model')
#model = joblib.load('_2017_1_24_knn.model')

In [ ]:
model = joblib.load('./model_check_point/knn_classifier.model')
predict = model.predict(X_test) 
accuracy = metrics.accuracy_score(y_test, predict)  
print ('accuracy: %.2f%%' % (100 * accuracy)  ) 